<a href="https://colab.research.google.com/github/Prakash-Nagamani/summer_intern/blob/main/movie_recommendation_system_using_cosine_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Importing Dependencies and Dataset**

In [ ]:
import pandas as pd
import numpy as np
import difflib
import matplotlib.pyplot as plt
from scipy import spatial
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score
%matplotlib inline

In [ ]:
movies_dataset = pd.read_csv("/content/movies_metadata.csv")
keywords_dataset = pd.read_csv("/content/keywords.csv")
credits_dataset = pd.read_csv("/content/credits.csv")

NameError: ignored

# **Data Pre-Processing** 

In [ ]:
movies_dataset.head(1)

NameError: ignored

In [ ]:
credits_dataset.head(1)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862


In [ ]:
movies_dataset.drop(['poster_path','spoken_languages','production_companies','original_language','release_date','production_countries','runtime','title','status'  ,'video','revenue','adult','belongs_to_collection','budget','homepage','imdb_id'], axis = 1,inplace=True)

In [ ]:
credits_dataset['crew'] = credits_dataset['crew'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x if i['job'] == 'Director'] if isinstance(x, list) else [])
credits_dataset.rename(columns = {'crew':'Director'}, inplace = True)

In [ ]:
index = movies_dataset.index
condition = movies_dataset['id'].str.contains(pat = '-')
wrong = index[condition]
wrong
movies_dataset.drop(wrong, inplace = True)
# movies_dataset['id'][35587]

In [ ]:
movies_dataset

,genres,id,original_title,overview,popularity,tagline,vote_average,vote_count
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,NaN,7.7,5415.0
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,Roll the dice and unleash the excitement!,6.9,2413.0
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,Still Yelling. Still Fighting. Still Ready for...,6.5,92.0
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,Friends are the people who let you be yourself...,6.1,34.0
4,"[{'id': 35, 'name': 'Comedy'}]",11862,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,Just When His World Is Back To Normal... He's ...,5.7,173.0
...,...,...,...,...,...,...,...,...
45461,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",439050,رگ خواب,Rising and falling between a man and woman.,0.072051,Rising and falling between a man and woman,4.0,1.0
45462,"[{'id': 18, 'name': 'Drama'}]",111109,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,0.178241,NaN,9.0,3.0
45463,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",67758,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,A deadly game of wits.,3.8,6.0
45464,[],227506,Satana likuyushchiy,"In a small town live two brothers, one a minis...",0.003503,NaN,0.0,0.0


Note: The above list "wrong" contains the indices of movies_dataset IDs that are in date format instead of integer

In [ ]:
# Converting JSON to List 
credits_dataset['cast'] = credits_dataset['cast'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
movies_dataset['genres'] = movies_dataset['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
keywords_dataset['keywords'] = keywords_dataset['keywords'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])


In [ ]:
# converting ID column type object to integer
movies_dataset['id'].dtypes
movies_dataset['id'] = movies_dataset['id'].astype(str).astype(int)

In [ ]:
# Merging all three datasets(credits_dataset,movies_dataset,keywords_dataset) using ID.
movies_dataset['id'] = movies_dataset['id'].astype(int)
movies = movies_dataset.merge(credits_dataset,on='id',how='left')
movies = movies.merge(keywords_dataset,on='id',how='left')
movies.head(1)


,genres,id,original_title,overview,popularity,tagline,vote_average,vote_count,cast,Director,keywords
0,"[Animation, Comedy, Family]",862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,NaN,7.7,5415.0,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",[John Lasseter],"[jealousy, toy, boy, friendship, friends, riva..."


In [ ]:
movies.head(1)

,genres,id,original_title,overview,popularity,tagline,vote_average,vote_count,cast,Director,keywords
0,"[Animation, Comedy, Family]",862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,NaN,7.7,5415.0,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",[John Lasseter],"[jealousy, toy, boy, friendship, friends, riva..."


In [ ]:
len(movies)

46629

In [ ]:
movies.dropna(subset=['vote_count'],inplace=True)

In [ ]:
zero_genres = []
for i,j in zip(movies.index,movies['genres']):
  if len(j) == 0:
    zero_genres.append(i)
movies.drop(zero_genres,inplace=True)
zero_votes = []
for i,j in zip(movies.index,movies['vote_count']):
  if j<7:
    zero_votes.append(i)
movies.drop(zero_votes,inplace=True)
zero_cast = []
for i,j in zip(movies.index,movies['cast']):
  if len(j) == 0:
    zero_cast.append(i)
movies.drop(zero_cast,inplace=True)

In [ ]:
len(movies)

26998

In [ ]:
movies['tagline'] = movies['tagline'].fillna('')

In [ ]:
selected_features = ['genres','keywords','tagline','cast','Director']
for feature in selected_features:
  movies[feature] = movies[feature].fillna('')

In [ ]:
movies.head(1)

,genres,id,original_title,overview,popularity,tagline,vote_average,vote_count,cast,Director,keywords
0,"[Animation, Comedy, Family]",862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,,7.7,5415.0,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",[John Lasseter],"[jealousy, toy, boy, friendship, friends, riva..."


In [ ]:
movies['genres'] = [' '.join(map(str, l)) for l in movies['genres']]
movies['cast'] = [' '.join(map(str, l)) for l in movies['cast']]
movies['Director'] = [' '.join(map(str, l)) for l in movies['Director']]
movies['keywords'] = [' '.join(map(str, l)) for l in movies['keywords']]

In [ ]:
movies.drop_duplicates(subset ="original_title",keep = 'first', inplace = True)
movies.reset_index(inplace = True)

In [ ]:
movies.head(1)

,index,genres,id,original_title,overview,popularity,tagline,vote_average,vote_count,cast,Director,keywords
0,0,Animation Comedy Family,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,,7.7,5415.0,Tom Hanks Tim Allen Don Rickles Jim Varney Wal...,John Lasseter,jealousy toy boy friendship friends rivalry bo...


In [ ]:
combined_features = movies['genres']+' '+movies['keywords']+' '+movies['tagline']+' '+movies['cast']+' '+movies['Director']

In [ ]:
print(combined_features)

0        Animation Comedy Family jealousy toy boy frien...
1        Adventure Fantasy Family board game disappeara...
2        Romance Comedy fishing best friend duringcredi...
3        Comedy Drama Romance based on novel interracia...
4        Comedy baby midlife crisis confidence aging da...
                               ...                        
25451    Fantasy Comedy tenant silent film  André Méliè...
25452    Animation Family halloween winnie the pooh Cel...
25453    Fantasy Action Thriller   Georges Méliès Georg...
25454    Crime Drama Thriller physical abuse sexual ass...
25455    Horror Mystery Thriller revenge murder serial ...
Length: 25456, dtype: object


# **Creating Machine Learning Model Using Cosine Similarity**

In [ ]:
# converting the text data to feature vectors

vectorizer = TfidfVectorizer()
feature_vectors = vectorizer.fit_transform(combined_features)
print(feature_vectors)

  (0, 52460)	0.19673392137743623
  (0, 44990)	0.19130971117604414
  (0, 70972)	0.13767033501077203
  (0, 31986)	0.12425341618575414
  (0, 80759)	0.09976182985207929
  (0, 28385)	0.17271607995094562
  (0, 53066)	0.07719924455701299
  (0, 61187)	0.1700669072245603
  (0, 52637)	0.13651664767557395
  (0, 23720)	0.20124740101769625
  (0, 98075)	0.1125692037146398
  (0, 28336)	0.12621047363058546
  (0, 63470)	0.11903974079973112
  (0, 73304)	0.16945065026747316
  (0, 3646)	0.13072395327084546
  (0, 75835)	0.16659975446799952
  (0, 45230)	0.16912148116486417
  (0, 83724)	0.12030254646603378
  (0, 98593)	0.12071652082483593
  (0, 96511)	0.1844410757004759
  (0, 44999)	0.09392412766092077
  (0, 77276)	0.17918661703603694
  (0, 25099)	0.09274929131440056
  (0, 2404)	0.10225383488644063
  (0, 92681)	0.09465528864222073
  :	:
  (25455, 35590)	0.1596262129136785
  (25455, 65151)	0.1646630946111304
  (25455, 60408)	0.1596262129136785
  (25455, 4837)	0.13057252509420522
  (25455, 76992)	0.11628867827

**Summary of Above code** <br>
1. The fit method is calculating the mean and variance of each of the features present in our data.
2. The transform method is transforming all the features using the respective mean and variance.<br><br><br>
fit_transform() is used on the training data so that we can scale the training data and also learn the scaling parameters of that data. Here, the model built by us will learn the mean and variance of the features of the training set. These learned parameters are then used to scale our test data.<br>
The Above values will be used to check the similarity between 2 points(Movies).





**cosine similarity**
<br>
The cosine similarity is advantageous because even if the two similar documents are far apart by the Euclidean distance, chances are they may still be oriented closer together. The smaller the angle, higher the cosine similarity.

In [ ]:
# getting the similarity scores using cosine similarity

similarity = cosine_similarity(feature_vectors)

In [ ]:
print(similarity.shape)

(25456, 25456)


In [ ]:
# getting the movie name from the user

movie_name = input(' Enter your favourite movie name : ')

 Enter your favourite movie name : minions


In [ ]:
# creating a list with all the movie names given in the dataset

list_of_all_titles = movies['original_title'].tolist()
print(list_of_all_titles)


['Toy Story', 'Jumanji', 'Grumpier Old Men', 'Waiting to Exhale', 'Father of the Bride Part II', 'Heat', 'Sabrina', 'Tom and Huck', 'Sudden Death', 'GoldenEye', 'The American President', 'Dracula: Dead and Loving It', 'Balto', 'Nixon', 'Cutthroat Island', 'Casino', 'Sense and Sensibility', 'Four Rooms', 'Ace Ventura: When Nature Calls', 'Money Train', 'Get Shorty', 'Copycat', 'Assassins', 'Powder', 'Leaving Las Vegas', 'Othello', 'Now and Then', 'Persuasion', 'La Cité des Enfants Perdus', '摇啊摇，摇到外婆桥', 'Dangerous Minds', 'Twelve Monkeys', 'Babe', 'Carrington', 'Dead Man Walking', 'It Takes Two', 'Clueless', 'Cry, the Beloved Country', 'Richard III', 'Dead Presidents', 'Restoration', 'Mortal Kombat', 'To Die For', 'How To Make An American Quilt', 'Se7en', 'Pocahontas', 'When Night Is Falling', 'The Usual Suspects', 'Mighty Aphrodite', 'Lamerica', 'The Big Green', 'Georgia', 'Home for the Holidays', 'Il postino', 'The Indian in the Cupboard', 'Eye for an Eye', "Mr. Holland's Opus", "Don't

In [ ]:
# finding the close match for the movie name given by the user

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

['Minions', 'Visions', 'Dimensions']


In [ ]:
close_match = find_close_match[0]
print(close_match)

Minions


In [ ]:
# finding the index of the movie with title

index_of_the_movie = movies[movies.original_title == close_match].index.values[0]
print(index_of_the_movie)

19609


In [ ]:
# getting a list of similar movies

similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

[(0, 0.012976825246275105), (1, 0.006836892214157915), (2, 0.014619860635807166), (3, 0.00818763226524847), (4, 0.039821573246959154), (5, 0.025218154747659854), (6, 0.015465360303290758), (7, 0.04770341110544239), (8, 0.005845405508144182), (9, 0.007520875135897231), (10, 0.012071257456042086), (11, 0.0023427800756281856), (12, 0.02466133692562072), (13, 0.012637753104430379), (14, 0.011427771408294471), (15, 0.011715707592567943), (16, 0.0), (17, 0.02067205181283569), (18, 0.018115111774165797), (19, 0.00907552409228065), (20, 0.009395842748563523), (21, 0.02535362913275869), (22, 0.021226691881577376), (23, 0.0), (24, 0.0), (25, 0.015016067149774784), (26, 0.008760871168615524), (27, 0.0), (28, 0.022247541338513102), (29, 0.0), (30, 0.0), (31, 0.02291395398475228), (32, 0.037302871489249774), (33, 0.017126787087486803), (34, 0.02807524095351651), (35, 0.018403478295656917), (36, 0.006058846600263791), (37, 0.00950133432302256), (38, 0.0), (39, 0.015258452396771583), (40, 0.007705539

In [ ]:
# sorting the movies based on their similarity score

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 
print(sorted_similar_movies)

[(19609, 1.0), (25353, 0.4072782841768693), (25398, 0.3630906259013125), (24958, 0.3032334958204824), (24778, 0.25157029641846923), (15279, 0.1985440175352517), (24066, 0.18397544678659347), (12177, 0.14905158921338554), (13905, 0.13048905679348027), (12776, 0.09899515866986988), (11418, 0.09632973768140692), (8679, 0.09231630476957738), (22593, 0.0893075845704799), (17992, 0.08867255065575533), (15244, 0.08505078444421316), (6353, 0.08472951390778515), (22823, 0.08382777169335526), (16246, 0.0833773754012269), (23657, 0.08297328301490857), (22592, 0.08283252804184767), (14498, 0.08182943277070545), (18152, 0.08141752088731838), (11602, 0.07974451301432015), (17352, 0.07957067685802115), (13852, 0.07935003483666052), (10896, 0.07922709337139838), (1897, 0.07849647096252721), (5183, 0.07805004099970542), (19398, 0.0773261917924221), (13672, 0.07720261596526973), (15579, 0.07706743954594286), (9790, 0.07467335476056176), (14614, 0.07458312225391933), (10161, 0.07451118319714739), (12339,

In [ ]:
# print the name of similar movies based on the index

print('Movies suggested for you : \n')
i = 1
for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies[movies.index==index]['original_title'].values
  rating_from_index = movies[movies.index==index]['vote_average'].values
  if len(title_from_index) != 0 and rating_from_index[0]>6.5:
    if (i<16):
      print(i, '.',title_from_index[0],'| Rating:',rating_from_index[0])
      i+=1
    else:
      break

Movies suggested for you : 

1 . Banana | Rating: 7.2
2 . Minions: Orientation Day | Rating: 7.0
3 . Despicable Me 2 | Rating: 7.0
4 . Mower Minions | Rating: 6.8
5 . Despicable Me | Rating: 7.1
6 . The Electric House | Rating: 7.1
7 . The White Countess | Rating: 6.9
8 . Monsters University | Rating: 7.0
9 . Mr. Peabody & Sherman | Rating: 6.7
10 . Wreck-It Ralph | Rating: 7.1
11 . Young Sherlock Holmes | Rating: 6.7
12 . Finding Nemo | Rating: 7.6
13 . Justice League: The New Frontier | Rating: 6.6
14 . Coraline | Rating: 7.3
15 . The Goat | Rating: 7.0


In [ ]:
def Recommend_movie(movie_name):
  # movie_name = input(' Enter your favourite movie name : ')
  find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
  print("Do you mean?")
  for ind in range(len(find_close_match)):
    print(ind+1,".",find_close_match[ind])
  req = int(input("Please select Number: "))
  close_match = find_close_match[req-1]
  index_of_the_movie = movies[movies.original_title == close_match].index.values[0]
  similarity_score = list(enumerate(similarity[index_of_the_movie]))
  sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)
  print('Movies suggested for you : \n')
  i = 0
  for movie in sorted_similar_movies:
    index = movie[0]
    title_from_index = movies[movies.index==index]['original_title'].values
    rating_from_index = movies[movies.index==index]['vote_average'].values
    if len(title_from_index) != 0 and rating_from_index[0]>6.5:
      if (i<15):
        print(i+1, '.',title_from_index[0],'| Rating:',rating_from_index[0])
        i+=1
      else:
        break


In [ ]:
Recommend_movie(input('Search movies : '))

Search movies : dilwale
Do you mean?
1 . Dilwale
Please select Number: 1
Movies suggested for you : 

1 . Main Hoon Na | Rating: 7.1
2 . Baazigar | Rating: 6.8
3 . भूतनाथ रिटर्न्स | Rating: 6.7
4 . Karan Arjun | Rating: 6.6
5 . Don 2: The King is Back | Rating: 7.1
6 . कल हो ना हो | Rating: 7.3
7 . Veer-Zaara | Rating: 7.5
8 . दिल से | Rating: 6.7
9 . Kuch Kuch Hota Hai | Rating: 7.5
10 . Om Shanti Om | Rating: 7.2
11 . Luck By Chance | Rating: 6.6
12 . Dilwale Dulhania Le Jayenge | Rating: 9.1
13 . Dear Zindagi | Rating: 6.9
14 . Dhobi Ghat | Rating: 6.7
15 . Don : The Chase Begins Again | Rating: 7.3


In [ ]:
Recommend_movie(input('Search movies : '))

Search movies : sweet
Do you mean?
1 . Sweetie
2 . Teeth
Please select Number: 2
Movies suggested for you : 

1 . Y tu mamá también | Rating: 7.3
2 . Dating The Enemy | Rating: 6.7
3 . The Disappearance of Eleanor Rigby: Her | Rating: 6.6
4 . Rounders | Rating: 6.9
5 . Sister Cities | Rating: 7.5
6 . Eddie Murphy Raw | Rating: 7.4
7 . Little Manhattan | Rating: 6.9
8 . Mommy Dead and Dearest | Rating: 7.2
9 . Sudden Fear | Rating: 7.1
10 . Korgoth of Barbaria | Rating: 8.5
11 . A New Leaf | Rating: 7.1
12 . Lamb | Rating: 6.8
13 . 喜宴 | Rating: 6.8
14 . Twin Peaks: Fire Walk with Me | Rating: 7.3
15 . Watching the Detectives | Rating: 6.6
